### Stock price performance pull

To get stock data, want to use the creation date of the file to then link to the stocks price. Also, it will be helpful to get data about SPY, QQQ, DIA, and ETF's related to specific industries. Some price performance measures to track are:
- 1M
- 3M
- 6M
- 1Y
- YTD

In [8]:
MONTH = 2
YEAR = 2022

In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
import pathlib
import os
#import datetime
#dt = datetime.datetime.today()
import time
from yahoofinancials import YahooFinancials
import csv
from csv import writer
from tqdm import tqdm
import os
import time
from datetime import datetime, timedelta

ESG_FILE_PATH = f"/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/esg-{MONTH}-{YEAR}.csv"

USER_PATH = '/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/'
DATE_PATH = "esg-{}-{}.csv".format(MONTH, YEAR)
FILE_PATH = os.path.join(USER_PATH,DATE_PATH)

1. Download esg data files from folder and save as pandas dataframe
2. Save tickers into a new dataframe
3. Loop through tickers, pull stock price data, calculate measures and save in esg file


In [10]:
print(ESG_FILE_PATH)

/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/esg-2-2022.csv


In [11]:
stat = os.stat(ESG_FILE_PATH)
stat.st_birthtime

1645317722.7837439

In [12]:
#need to get creation time not change time
def get_ctime(path):
    #c_time = pathlib.Path(path).stat().st_ctime this is for windows
    c_time = os.stat(ESG_FILE_PATH)
    c_time = c_time.st_birthtime
    c_time = time.strftime('%Y-%m-%d', time.localtime(c_time)) 
    c_time = datetime.fromisoformat(c_time)
    return c_time

In [13]:
x = get_ctime(ESG_FILE_PATH)
print(x)

2022-02-19 00:00:00


In [14]:
#esg_data = pd.read_csv(FILE_PATH) #add in drop duplic
#esg_data = esg_data.drop_duplicates() #Ensure no duplicates
#tickers = esg_data['Stock Ticker']

In [15]:
df = pd.read_csv(ESG_FILE_PATH)

tickers = df[['Stock Ticker']].copy()
tickers = tickers.drop_duplicates()

esg_ctime = get_ctime(ESG_FILE_PATH)
one_year_ctime = esg_ctime - timedelta(365)

print(esg_ctime)

2022-02-19 00:00:00


In [16]:
#ticker_sym = tickers['Stock Ticker'].loc[0] # get ticker name ex: 'AAPL'
#print(ticker_sym)
#len = tickers.shape[0]
#print(len)

In [17]:
def get_stocks(t_df):
    price_df = pd.DataFrame(columns=["Stock Ticker","One Year","YTD","3-Month","6-Month"])
    for x in range(3):
        ticker_sym = ''

        ticker_sym = t_df['Stock Ticker'].head()
        
        print(ticker_sym[0])
        stock_df = yf.download(ticker_sym[0], 
                        start=one_year_ctime, 
                        end=esg_ctime, 
                        progress=False,
        )
        stock_df.reset_index(inplace=True)
        stock_df = stock_df.rename(columns = {'Date':'Date'})
        size = stock_df.shape[0]
        six_month = int(size / 2)

        first_price = stock_df['Close'].head(1).values[0]
        last_price = stock_df['Close'].tail(1).values[0]
        one_year_perf = (last_price / first_price) - 1

        six_loc = int(stock_df.shape[0]/2)
        half_price = stock_df['Close'].loc[six_loc]
        six_month_perf = (last_price / half_price) -1

        three = int(stock_df.shape[0]*3/4)
        three_price = stock_df['Close'].loc[three]
        three_month_perf = (last_price / three_price) - 1

        stock_df['Date'] = pd.DatetimeIndex(stock_df['Date']).year
        stock_df = stock_df[stock_df['Date'] == YEAR]

        first_price = stock_df['Close'].head(1).values[0]
        last_price = stock_df['Close'].tail(1).values[0]
        ytd = (last_price / first_price) - 1

        df2 = {'Stock Ticker': ticker_sym[0], 'One Year': one_year_perf, 'YTD': ytd, '3-Month':three_month_perf, '6-Month':six_month_perf}
        price_df = price_df.append(df2, ignore_index = True)
        del df2 
        del stock_df 

        t_df = t_df.iloc[1: , :]
        t_df.reset_index(inplace=True)

    return price_df

#dfff = get_stocks(tickers)

In [18]:
tickers

,Stock Ticker
0,DPZ
1,APA
2,MSCI
3,EMN
4,NFLX
...,...
498,PM
499,VNO
501,CVS
502,NLSN


In [19]:
# iterating over rows using iterrows() function
large_str = ""
x = 0
for i, j in tickers.iterrows():
    large_str += j + " "
    x += 1
print(x)

288


In [20]:
large_str[0]

'DPZ APA MSCI EMN NFLX REG EOG MMM SHW WAT HAL KLAC CTSH AZO FTV AMZN CHRW GE MNST DRI BXP PPG BK VLO CTRA AVB CMI ZTS DGX BIIB TT DIS GRMN AMGN AAL TPR WMT ATVI NVR ULTA COF LUV NTRS PYPL FIS AEE KEY BLK MSI XYL BEN NDAQ COST KMB CMS TSCO GPC SYY ACN KR SCHW IP PEG LNT OMC HST HIG LEN CVX APH QRVO NLOK PG BRO ESS GOOGL PNW ITW LKQ IDXX NI FITB AOS HON FMC ICE MMC SBAC BBWI DRE KIM DVN PEP PTC DXCM EMR AVY MPC CRM FCX LNC PBCT ATO FISV TFX NUE INFO SWK AKAM AEP UAL IPGP GPN CHD TRMB SBUX ILMN LEG XLNX MTB ALLE KO ABT CMCSA CDW PHM NOW XEL CSX EQR TWTR MA SPG MTD UAA ROK VRSK DISCA DAL CE PNR EBAY DTE PXD JCI BIO HOLX NCLH HUM TSLA WFC FRT HBAN LUMN LYB WHR VTR PH NEE TMO L PFG GL J CBOE WY ETR ADP CME CPRT NSC TGT MCK NWSA TROW UPS CI AXP BR IFF XOM EXR KMX AVGO BAX PKG PCAR HPQ FBHS NKE NVDA LVS JNPR CF FRC NXPI AME WEC VRSN FTNT DLR PAYX VRTX SNPS CAG LH KEYS MCO GIS TEL ECL STX DUK APTV DD BA ZBH ADM OXY JBHT JKHY CTAS MDT CDNS TSN MOS INTC WLTW VZ BDX RE CINF PNC NTAP SRE GILD URI 

In [38]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = large_str[0],
        
        #"SPY AAPL MSFT GOOG ENPH V",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "2y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  288 of 288 completed

5 Failed downloads:
- XLNX: No data found, symbol may be delisted
- WLTW: No data found, symbol may be delisted
- PBCT: No data found, symbol may be delisted
- DISCA: No data found, symbol may be delisted
- INFO: No data found, symbol may be delisted


In [39]:
data
data.reset_index(inplace=True)
data = data.rename(columns = {'Date':'Date'})

In [52]:
test = data
test # need to get dates set to the time of the ESG data pull.

Date          PH                                               \
                      Open        High         Low       Close   Volume   
0   2020-06-23  180.529988  180.792136  176.073540  176.287140   972200   
1   2020-06-24  174.277386  174.471564  167.277157  168.665558  1764400   
2   2020-06-25  168.248071  171.675379  165.345076  171.481186   948200   
3   2020-06-26  170.869500  171.452033  166.587807  167.519867  1482500   
4   2020-06-29  169.694696  174.976419  168.549036  174.345337   841800   
..         ...         ...         ...         ...         ...      ...   
499 2022-06-15  254.929993  258.079987  251.130005  254.789993   665000   
500 2022-06-16  248.500000  248.500000  238.259995  240.539993   980100   
501 2022-06-17  232.990005  239.360001  230.440002  236.369995  1518100   
502 2022-06-21  240.179993  242.410004  236.449997  241.899994   925500   
503 2022-06-22  237.190002  245.860001  237.190002  243.869995   832400   

           JCI                                   ...        ICE             \
          Open       High        Low      Close  ...       Open       High   
0    34.016661  34.016661  33.153049  33.383347  ...        NaN        NaN   
1    33.057094  33.210625  31.867229  31.943996  ...  91.375966  91.710320   
2    31.838443  32.135910  31.377851  32.107121  ...  89.350209  91.415313   
3    31.905613  32.097527  31.483404  32.011166  ...  90.608927  91.248128   
4    32.087931  32.548523  31.780870  32.452568  ...  89.773053  89.773053   
..         ...        ...        ...        ...  ...        ...        ...   
499  49.670274  50.226134  48.766998  49.491604  ...  94.830002  96.930000   
500  48.350096  48.409654  46.662656  47.019997  ...  93.720001  95.239998   
501  46.889999  48.029999  46.549999  47.230000  ...  93.930000  94.699997   
502  48.110001  48.189999  47.150002  47.400002  ...  91.570000  94.120003   
503  46.740002  47.959999  46.630001  47.439999  ...  92.550003  95.199997   

                                             CAG                        \
           Low      Close      Volume       Open       High        Low   
0          NaN        NaN         NaN        NaN        NaN        NaN   
1    88.947008  89.202690   3967700.0  32.133050  32.161401  31.603798   
2    88.514335  91.287476   2718900.0  31.972394  32.066902  31.736121   
3    88.829006  88.897842   3368700.0  31.934585  32.000741  30.791028   
4    88.602824  88.897842   2186600.0  31.159608  32.047992  31.074550   
..         ...        ...         ...        ...        ...        ...   
499  93.620003  95.629997   3087500.0  31.930000  32.330002  31.620001   
500  93.349998  94.849998   2618800.0  31.719999  32.029999  31.100000   
501  90.050003  90.330002  10178000.0  31.940001  32.110001  31.070000   
502  91.220001  93.809998   4550500.0  31.660000  32.480000  31.490000   
503  92.279999  93.959999   2835100.0  32.410000  32.580002  32.040001   

                           
         Close     Volume  
0          NaN        NaN  
1    31.934582  3774100.0  
2    31.840078  3862700.0  
3    30.847731  5568000.0  
4    31.849524  4521700.0  
..         ...        ...  
499  31.980000  4491700.0  
500  31.750000  3775800.0  
501  31.370001  6369400.0  
502  32.340000  3567000.0  
503  32.459999  3216100.0  

[504 rows x 1446 columns]

In [131]:
index = pd.MultiIndex.from_tuples(test, names=['Tickers','Cols'])
t = index.get_level_values(0)
t = t.to_frame(index=False).drop_duplicates().reset_index(drop=True)
t = t.iloc[1:,:]

In [140]:
t['Tickers'].head()

1      PH
2     JCI
3     IPG
4    INTC
5     UAA
Name: Tickers, dtype: object

In [150]:
data['PH']['Close']

0      176.287140
1      168.665558
2      171.481186
3      167.519867
4      174.345337
          ...    
499    254.789993
500    240.539993
501    236.369995
502    241.899994
503    243.869995
Name: Close, Length: 504, dtype: float64

In [156]:
t.iloc[0]['Tickers']

'PH'

In [166]:
s = t.shape[0]
for i in range(s): 
    t_name = t.iloc[i]['Tickers']
    first_price = data[t_name]['Close'].head(1).values[0]
    last_price = data[t_name]['Close'].tail(1).values[0]
    one_year_perf = (last_price / first_price) - 1
    print(f"{t_name} 1-year is {one_year_perf}")

PH 1-year is 0.38336803958468835
JCI 1-year is 0.4210677933331535
IPG 1-year is 0.7037694346191179
INTC 1-year is -0.3408453897083431
UAA 1-year is -0.09432042432898713
EOG 1-year is 1.3590204574377815
COF 1-year is 0.563165849093926
AVB 1-year is 0.3211547013511584
CTSH 1-year is 0.25426361173205736
BEN 1-year is 0.21559130066105792
KMB 1-year is -0.030019309286080875
TGT 1-year is 0.19557764001871591
XLNX 1-year is nan
MCK 1-year is 1.0594082251776311
BRO 1-year is 0.36579205374544776
AVY 1-year is 0.38952562143212455
MCO 1-year is -0.034583379513169366
PM 1-year is 0.525133149914538
APA 1-year is 1.7014894549327861
VNO 1-year is -0.1996327408575551
ATO 1-year is 0.1250932834153664
WLTW 1-year is nan
AXP 1-year is 0.4900225769036606
SYY 1-year is 0.41318537598352467
JKHY 1-year is -0.029044000486370236
BK 1-year is 0.15509084608789458
SNPS 1-year is 0.5618810486895178
COST 1-year is 0.5899987899864894
HON 1-year is 0.2742593595222984
ZTS 1-year is 0.20273239482583305
PNW 1-year is 0.

In [27]:
data.drop(['High', 'Low','Volume'], axis=1,level=1)
data[tickers.head().iloc[0]]
data.reset_index(inplace=True)
data = data.rename(columns = {'Date':'Date'})


size = data.shape[0]
six_month = int(size / 2)

first_price = data['Close'].head(1).values[0]
last_price = data['Close'].tail(1).values[0]
one_year_perf = (last_price / first_price) - 1

six_loc = int(data.shape[0]/2)
half_price = data['Close'].loc[six_loc]
six_month_perf = (last_price / half_price) -1

three = int(data.shape[0]*3/4)
three_price = data['Close'].loc[three]
three_month_perf = (last_price / three_price) - 1

data['Date'] = pd.DatetimeIndex(data['Date']).year
data = data[data['Date'] == YEAR]

first_price = data['Close'].head(1).values[0]
last_price = data['Close'].tail(1).values[0]
ytd = (last_price / first_price) - 1

KeyError: 'Close'

In [ ]:
six_month

In [ ]:
ticker = 'AAPL'
stock_df = yf.download(ticker, 
                      start=one_year_ctime, 
                      end=esg_ctime, 
                      progress=False,
)
stock_df.reset_index(inplace=True)
stock_df = stock_df.rename(columns = {'Date':'Date'})
stock_df.head()
size = stock_df.shape[0]
six_month = int(size / 2)

first_price = stock_df['Close'].head(1).values[0]
last_price = stock_df['Close'].tail(1).values[0]
one_year_perf = (last_price / first_price) - 1

six_loc = int(stock_df.shape[0]/2)
half_price = stock_df['Close'].loc[six_loc]
six_month_perf = (last_price / half_price) -1

three = int(stock_df.shape[0]*3/4)
three_price = stock_df['Close'].loc[three]
three_month_perf = (last_price / three_price) - 1

stock_df['Date'] = pd.DatetimeIndex(stock_df['Date']).year
stock_df = stock_df[stock_df['Date'] == YEAR]

first_price = stock_df['Close'].head(1).values[0]
last_price = stock_df['Close'].tail(1).values[0]
ytd = (last_price / first_price) - 1

In [ ]:
stock_df.tail()

In [ ]:
price_df = pd.DataFrame(columns=["Stock Ticker","One Year","YTD"])
price_df

In [ ]:
first_price = stock_df['Close'].head(1).values[0]
last_price = stock_df['Close'].tail(1).values[0]
one_year = (last_price / first_price) - 1

print(first_price)
print(last_price)
print(one_year)

six_loc = int(stock_df.shape[0]/2)
half_price = stock_df['Close'].loc[six_loc]

three = int(stock_df.shape[0]*3/4)
three_price = stock_df['Close'].loc[three]

three_month = (last_price / three_price) - 1
six_month = (last_price / half_price) - 1

stock_df['Date'] = pd.DatetimeIndex(stock_df['Date']).year
stock_df = stock_df[stock_df['Date'] == YEAR]

first_price = stock_df['Close'].head(1).values[0]
last_price = stock_df['Close'].tail(1).values[0]
ytd = (last_price / first_price) - 1

df2 = {'Stock Ticker': ticker, 'One Year': one_year, 'YTD': ytd, '3-Month':three_month, '6-month':six_month}
price_df = price_df.append(df2, ignore_index = True)
print(first_price)
print(last_price)
print(ytd)

In [ ]:

price_df

In [ ]:
stock_df.head()

In [ ]:
last_price

In [ ]:
#yahoo_financials = YahooFinancials('AAPL')
#x = yahoo_financials.get_summary_data()
#y = yahoo_financials.get_financial_stmts('annual', 'income', reformat=True)
#x.keys()